# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(
    base_url = "https://api.xiaoai.plus/v1"
)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图描绘了一条蜿蜒于郁郁葱葱草地中的木板路。木板路直通天际，引导观者的视线深入画面。四周的大自然景观充满活力，草地以不同的绿色层次铺展，而天空则呈现出宁静的蓝色和轻柔的云朵。\n\n画面左侧和右侧各有一些树木和灌木丛，为画面添加了一些复杂性和深度。这种设置很可能是在一个自然保护区或是乡村公园中，这样的木板路通常用于保护地面植被，同时提供人们观赏和接近自然的途径，而不破坏周围的环境。\n\n整体上，这幅图表现了一种平和与和谐的氛围，引人深思地感受户外的宁静与自然之美。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图描绘了一条蜿蜒于郁郁葱葱草地中的木板路。木板路直通天际，引导观者的视线深入画面。四周的大自然景观充满活力，草地以不同的绿色层次铺展，而天空则呈现出宁静的蓝色和轻柔的云朵。\n\n画面左侧和右侧各有一些树木和灌木丛，为画面添加了一些复杂性和深度。这种设置很可能是在一个自然保护区或是乡村公园中，这样的木板路通常用于保护地面植被，同时提供人们观赏和接近自然的途径，而不破坏周围的环境。\n\n整体上，这幅图表现了一种平和与和谐的氛围，引人深思地感受户外的宁静与自然之美。'

### 封装成一个函数 query_image_description

In [5]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(
        base_url = "https://api.xiaoai.plus/v1"
    )  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [6]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默搞笑的图，它对比了两种类型的“工作犬”。左边是一只肌肉非常发达、类似人类健美运动员的犬体，上面写的是“16岁的狗，工作后的狗”，具有超级英雄般的体格，还开玩笑地加了几个特点，比如“我刚逝世一片落叶、身体森林瘦”。右边是一只看起来比较懒惰、体型略显肥胖的犬体，上面写的是“工作后的狗”，形容它的状况是“好累好累，好想躺着流沙发”，显得很无奈但幽默。

这种幽默主要是通过夸张和人格化的手法来制造娱乐效果，让人们在笑话中反思现代工作压力与生活方式的对比。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [16]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(
    base_url = "https://api.xiaoai.plus/v1"
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [17]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图显示的是1980年到2020年间美国、中国、日本和德国的国内生产总值（GDP）的比较。图中使用不同颜色代表不同国家的GDP增长轨迹：蓝色代表美国，红色代表中国，紫色代表日本，绿色代表德国。

从图中可以观察到以下几点：

1. 美国（蓝色线）的GDP一直保持在较高水平，并在这40年间持续稳定增长，2020年接近20万亿美元。

2. 中国（红色线）的GDP增长非常迅猛，尤其是从2000年之后，增长速度显著加快，到2020年几乎赶上美国。

3. 日本（紫色线）的GDP在1990年代初达到高峰后，出现波动并相对平稳，在这四个国家中排在第三。

4. 德国（绿色线）的GDP增长相对平稳，维持在较低的增长速度，到2020年处于这四个国家中的第四位。

这幅图有效地展示了过去几十年中这四个经济体的经济增长情况，特别是突出了中国的快速崛起。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [18]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图是一张展示机器学习或自然语言处理领域中提示（Prompt）学习和模型调整技术的笔记图片。下面是图片中内容的逐段解释：

1. 首段提到“提示的PET方法”，PET可能代表“Prompt Engineering for Transformers”，这是一种使用预设语句来优化和改善转换器模型（如BERT、GPT等）的性能的技术。

2. 接下来的内容讲述了“Prompt Tuning (FMT & small model)”：
   - 这里的FMT可能是指一种用于小模型的训练方法。
   - 笔记解释了X的定义，X=[X1, X2, ..., Xn]是将输入变量通过Embedding转换后的序列，然后透过某种形式变成新的序列X'。
   - 式中Y=W*X'解释了通过权重矩阵W来转换输入X'得到输出Y的过程。
   
3. “Prefix Tuning”部分：
   - 介绍了如何使用权重W（由Wp和Wl组成）和Transformer的encoder/decoder来生成输出Y。
   
4. “LoRA（低秩适应）”部分：
   - 描述了低秩矩阵适应的概念，其中ΔW = AB是增加到原始权重矩阵W的低秩更新。
   - ΔW的形状是mxr和rxn，这是一种有效调整大型模型的方法，因其降低了参数数量并保持了相对高的表现力。
   - 提及了进阶的概念Q-LORA（可能是LoRA的一个变种或优化），以及与方法相关的模型性能和存储需求（如LAMA-65GB与Q-LORA-48GB的比较）。

这些笔记为自然语言处理领域中现代机器学习模型调整技术提供了理论基础和应用概览。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [12]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这幅图是一张幽默搞笑的图，它对比了两种类型的“工作犬”。左边是一只肌肉非常发达、类似人类健美运动员的犬体，上面写的是“16岁的狗，工作后的狗”，具有超级英雄般的体格，还开玩笑地加了几个特点，比如“我刚逝世一片落叶、身体森林瘦”。右边是一只看起来比较懒惰、体型略显肥胖的犬体，上面写的是“工作后的狗”，形容它的状况是“好累好累，好想躺着流沙发”，显得很无奈但幽默。

这种幽默主要是通过夸张和人格化的手法来制造娱乐效果，让人们在笑话中反思现代工作压力与生活方式的对比。

![](./images/handwriting_1.jpg)

In [ ]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [20]:
from openai import OpenAI
import base64
import requests
import json
from IPython.display import display, Markdown

client = OpenAI(
    base_url = "https://api.xiaoai.plus/v1"
)  # 初始化 OpenAI 客户端

def query_base64_image_description_with_markdown(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        # 使用 display 和 Markdown 函数显示 Markdown 内容
        display(Markdown(content))
    else:
        return f"Error: {response.status_code}, {response.text}"

In [21]:
query_base64_image_description_with_markdown("./images/handwriting_0.jpg")

这张图片展示的是一本笔记本上记录的关于机器学习中一些高级技术的笔记。内容涵盖了Prompt Tuning、Prefix Tuning和LoRA技术，这些都是用于提升深度学习模型（尤其是自然语言处理模型）性能的方法。下面是对图片中笔记的详细解释：

1. **Prompt Tuning (FMT & Small Model)**:
   - 提到了输入 \( X = [X_1, X_2, ..., X_n] \)，这些是经过embedding后的输入向量。
   - 输出 \( Y \) 是通过矩阵 \( W \) 与输入 \( X' \) 的乘积得到。这里 \( X' \) 可能表示经transformation后的输入。

2. **Prefix Tuning**:
   - 这种方法涉及到一个前缀权重 \( W' = [W_P : W_L] \)，其中 \( W_P \) 是前缀权重部分，通过对 \( X' \) 和权重 \( W' \) 的乘积得到输出 \( Y \)，与之前的Prompt Tuning类似但加入了前缀权重。

3. **LoRA (Linear Transformer)**:
   - 表示通过调整线性变换 \( W \) 来改变网络输出。增量 \( \Delta W = AB \) 是一种低秩更新，其中 \( \Delta W \) 是一个由 \( A \)（mxr）和 \( B \)（rxn）矩阵乘积产生的，用于微调已有的权重矩阵 \( W \)。（此处 \( m, r, n \) 为矩阵维度）
   - 学习目标是 \( (W+\Delta W)X \)。

4. **Q-LORA (可能是一种特殊的LoRA技术)**:
   - 提到了与LoRA类似的应用，但具体细节和区别没有提及。

5. **容量比较**:
   - 提到LAMA使用LoRA需要78GB的参数量，而使用QLoRA则只需48GB，这可能暗示QLoRA是一种更为高效或压缩的技术版本。

图片中的笔记不仅展示了这些技术的理论基础，还可能含有一些个人的理解和解释，适用于研究或进一步探讨自然语言处理领域中的模型优化。